# Семинар 7: Оценка качества моделей, кросс-валидация, подбор метапараметров

На предыдущем семинаре мы рассмотрели основной рецепт применения модели машинного обучения под наблюдением:

1. Выберите класс модели
2. Выбрать гиперпараметры модели
3. Подогнать модель к обучающим данным
4. Используйте модель для предсказания меток для новых данных.

Первые две части - выбор модели и выбор гиперпараметров - являются, пожалуй, самой важной частью эффективного использования этих инструментов и методов.
Чтобы сделать обоснованный выбор, нам нужен способ *подтвердить*, что наша модель и наши гиперпараметры хорошо подходят к данным.
Хотя это может показаться простым, есть несколько подводных камней, которых следует избегать, чтобы сделать это эффективно.

### Оценка качества моделей: неправильный подход

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

Выбираем модель и гиперпараметры. Здесь мы будем использовать *KNeighborsClassifier* с ``n_neighbors=1``.
Это очень простая и интуитивно понятная модель, которая говорит, что "метка неизвестной точки совпадает с меткой ее ближайшей обучающей точки:".

In [7]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [8]:
model.fit(X, y)
y_model = model.predict(X)

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

1.0

Мы видим оценку точности 1,0, что означает, что 100% точек были правильно помечены нашей моделью!
Но действительно ли это измерение ожидаемой точности? 
Действительно ли мы нашли модель, которая, как мы ожидаем, будет правильной в 100% случаев?

Как вы уже догадались, ответ отрицательный.
На самом деле, этот подход содержит фундаментальный недостаток: *он обучает и оценивает модель на одних и тех же данных*.
Более того, модель ближайшего соседа - это *оценка на основе случая*, которая просто хранит обучающие данные и предсказывает метки, сравнивая новые данные с этими сохраненными точками: за исключением надуманных случаев, она будет давать 100% точность *каждый раз*.

### Оценка качества моделей: валидация на отложенной выборке

Лучшее представление о производительности модели можно получить, используя так называемый *холд-аут набор*: то есть, мы удерживаем некоторое подмножество данных от обучения модели, а затем используем этот холд-аут набор для проверки производительности модели.
Такое разбиение можно сделать с помощью утилиты ``train_test_split`` в Scikit-Learn:

In [19]:
from sklearn.model_selection import train_test_split
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)

0.9066666666666666

### Оценка качества моделей: перекрестная валидация

Одним из недостатков использования набора данных для проверки модели является то, что мы потеряли часть данных для обучения модели.
В приведенном выше случае половина набора данных не участвует в обучении модели!
Это не оптимально и может вызвать проблемы - особенно если начальный набор обучающих данных невелик.

Одним из способов решения этой проблемы является использование *перекрестной валидации*; то есть, проведение последовательности подгонок, в которых каждое подмножество данных используется и как обучающий набор, и как проверочный набор.
Визуально это может выглядеть примерно так:

![](images/05.03-2-fold-CV.png)

Здесь мы проводим два проверочных испытания, поочередно используя каждую половину данных в качестве промежуточного набора.
Используя разделенные данные, мы могли бы реализовать это следующим образом:

In [13]:
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

(0.96, 0.9066666666666666)

В результате получаются две оценки точности, которые мы можем объединить (например, взяв среднее значение), чтобы получить более точную оценку эффективности глобальной модели.
Эта конкретная форма кросс-валидации является *двухкратной кросс-валидацией*, то есть, когда мы разделили данные на два набора и использовали каждый из них по очереди в качестве проверочного набора.

Мы можем расширить эту идею, чтобы использовать еще больше испытаний и больше складок в данных - например, вот визуальное изображение пятикратной кросс-валидации:

![](images/05.03-5-fold-CV.png)

Здесь мы разбиваем данные на пять групп и используем каждую из них по очереди для оценки соответствия модели на остальных 4/5 данных.
Это было бы довольно утомительно делать вручную, поэтому мы можем использовать удобную процедуру Scikit-Learn ``cross_val_score'', чтобы сделать это лаконично:

In [15]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [17]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
scores.mean()

0.96

### Выбор лучшей модели

Теперь, когда мы рассмотрели основы валидации и перекрестной валидации, мы немного углубимся в выбор модели и подбор гиперпараметров.
Эти вопросы являются одними из самых важных аспектов практики машинного обучения, и я обнаружил, что в вводных учебниках по машинному обучению эта информация часто упускается.

Основное значение имеет следующий вопрос: *если наша оценка не оправдывает себя, как мы должны двигаться дальше?
Есть несколько возможных ответов:

- Использовать более сложную/более гибкую модель
- Использовать менее сложную/менее гибкую модель
- Собрать больше обучающих выборок
- Собрать больше данных, чтобы добавить характеристики к каждой выборке.

Ответ на этот вопрос часто бывает контринтуитивным.
В частности, иногда использование более сложной модели дает худшие результаты, а добавление большего количества обучающих выборок может не улучшить ваши результаты!
Способность определить, какие шаги улучшат вашу модель, - это то, что отделяет успешных практиков машинного обучения от неуспешных.

### The Bias-variance trade-off. Дилемма смещения и дисперсии

![](images/05.03-validation-curve.png)